In [11]:
import torch, torchvision
from torch.utils.data import Dataset
from transformers import OwlViTProcessor, OwlViTForObjectDetection
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from torchvision.utils import draw_bounding_boxes
import torchvision.transforms as transforms




In [7]:
processor = OwlViTProcessor.from_pretrained("C:/Projects/ObjectDetection/transformers/owlvit-base-patch32")
model = OwlViTForObjectDetection.from_pretrained("C:/Projects/ObjectDetection/transformers/owlvit-base-patch32")

ftfy or spacy is not installed using BERT BasicTokenizer instead of ftfy.


In [18]:
image = Image.open("C:\Projects\ObjectDetection\data\order_images\order_photo_1.jpeg")
texts = [[ "food item", "a grocery item", "a packet", "fruits"]]
inputs = processor(text=texts, images=image, return_tensors="pt")
outputs = model(**inputs)

# Target image sizes (height, width) to rescale box predictions [batch_size, 2]
target_sizes = torch.Tensor([image.size[::-1]])
# Convert outputs (bounding boxes and class logits) to COCO API
results = processor.post_process(outputs=outputs, target_sizes=target_sizes)

i = 0  # Retrieve predictions for the first image for the corresponding text queries
text = texts[i]
boxes, scores, labels = results[i]["boxes"], results[i]["scores"], results[i]["labels"]

score_threshold = 0.1
thresholded_boxes = []
for box, score, label in zip(boxes, scores, labels):
    box = [round(i, 2) for i in box.tolist()]
    if score >= score_threshold:
      thresholded_boxes.append(box)
      #print(f"Detected {text[label]} with confidence {round(score.item(), 3)} at location {box}")

In [ ]:
transforms.Normalize()

In [35]:
transform = transforms.Compose([
    transforms.PILToTensor()
])

img_tensor = transform(image)

In [36]:
detected_objects  = []
for detect in thresholded_boxes:
    box = torch.tensor(detect, dtype=int)
    object_tensor = (img_tensor[:, box[1]:box[3], box[0]:box[2]])/255
    detected_objects.append(object_tensor)
                     

C:\Users\mithu\AppData\Local\Temp\ipykernel_14772\2206010338.py:3: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  box = torch.tensor(detect, dtype=int)


In [50]:
template = Image.open("..\\data\\template_images\\paneer.jpeg").convert('RGB')
template_tensor = transform(template)/255

c:\Users\mithu\anaconda3\lib\site-packages\PIL\Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [51]:
template_tensor.min(), template_tensor.max()

(tensor(0.0235), tensor(0.9765))

In [38]:
barlow_model = torch.hub.load('facebookresearch/barlowtwins:main', 'resnet50')

Using cache found in C:\Users\mithu/.cache\torch\hub\facebookresearch_barlowtwins_main
c:\Users\mithu\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mithu\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [52]:
template_representation = barlow_model(template_tensor.unsqueeze(0))
object_representation = barlow_model(detected_objects[0].unsqueeze(0))

In [58]:
torch.dot(template_representation[0],object_representation[0])

tensor(1.3130, grad_fn=<DotBackward0>)